In [1]:
# setup main dataframe, initial column is ["SUBJECT_ID", "HADM_ID", "ADMITTIME", "DIAGNOSIS"] from ADMISSIONS
# SUBJECT_ID: Each patient has unique ID
# HADM_ID: Each admission has unique ID
# ADMITTIME: The date and time the patient was admitted to the hospital
# DIAGNOSIS: The primary diagnosis of the patient

import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 0)

# read ADMISSIONS.csv.gz
mainDF = pd.read_csv('../mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz', compression='gzip', usecols=["SUBJECT_ID", "HADM_ID", "ADMITTIME", "DIAGNOSIS"])

# Change the ADMITTIME only have date
mainDF["ADMITTIME"] = pd.to_datetime(mainDF["ADMITTIME"]).dt.date.astype(str)

# Dtop the rows if DIAGNOSIS is NaN
mainDF = mainDF.dropna(subset=["DIAGNOSIS"])

# Separate the DIAGNOSIS if there are multiple, and output to dataframe
data = []
for row in mainDF.itertuples():

  if "\\" in row.DIAGNOSIS:
    diagnosisString = row.DIAGNOSIS.split("\\")[0]
  else:
    diagnosisString = row.DIAGNOSIS
  
  if ";" in diagnosisString:
    diagnosisArray = diagnosisString.split(";")
  else:
    diagnosisArray = [diagnosisString]

  for d in diagnosisArray:
    data.append([row.SUBJECT_ID, row.HADM_ID, row.ADMITTIME, d])

diagnosisDF = pd.DataFrame(data, columns=["SUBJECT_ID", "HADM_ID", "ADMITTIME", "DIAGNOSIS"])


In [2]:
# Add the height and weight to the main dataframe by using the ["SUBJECT_ID", "HADM_ID", "ITEMID", "VALUE"] in CHARTEVENTS
# SUBJECT_ID: Each patient has unique ID
# HADM_ID: Each admission has unique ID
# ITEMID: weight index : 762, 763, 3723, 3580, 3581, 3582
#         height index : 920, 1394, 4187, 3486, 3485, 4188
# VALUE: the value of the ITEMID

# read CHARTEVENTS.csv.gz and drop the missing value, and change the CHARTTIME only have date
CHARTEVENTS = pd.read_csv('../mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz', compression='gzip', usecols=["SUBJECT_ID", "HADM_ID", "CHARTTIME", "ITEMID", "VALUE"]).dropna()
CHARTEVENTS["CHARTTIME"] = pd.to_datetime(CHARTEVENTS["CHARTTIME"]).dt.date.astype(str)

# create weightHeightDict with key = (SUBJECT_ID, HADM_ID, CHARTTIME) and value = [weight, height]
weightHeightDict = {}
for row in CHARTEVENTS.itertuples():
  key = (row.SUBJECT_ID, row.HADM_ID, row.CHARTTIME)

  if row.ITEMID == 762 or row.ITEMID == 763 or row.ITEMID == 3723 or row.ITEMID == 3580: # weight unit is alreayd in KG
    if key in weightHeightDict and weightHeightDict[key][0] == -1: weightHeightDict[key][0] = float(row.VALUE)
    else: weightHeightDict[key] = [float(row.VALUE), -1]
  
  elif row.ITEMID == 3581: # weight unit is in LB
    if key in weightHeightDict and weightHeightDict[key][0] == -1: weightHeightDict[key][0] = float(row.VALUE)*0.4536
    else: weightHeightDict[key] = [float(row.VALUE)*0.4536, -1]
  
  elif row.ITEMID == 3582: # weight unit is in OZ
    if key in weightHeightDict and weightHeightDict[key][0] == -1: weightHeightDict[key][0] = float(row.VALUE)*0.0283
    else: weightHeightDict[key] = [float(row.VALUE)*0.0283, -1]
  
  elif row.ITEMID == 920 or row.ITEMID == 1394 or row.ITEMID == 4187 or row.ITEMID == 3486: # height unit is in inches
    if key in weightHeightDict and weightHeightDict[key][1] == -1: weightHeightDict[key][1] = float(row.VALUE)*0.0254
    else: weightHeightDict[key] = [-1, float(row.VALUE)*0.0254]

  elif row.ITEMID == 3485 or row.ITEMID == 4188: # height unit is in cm
    if key in weightHeightDict and weightHeightDict[key][1] == -1: weightHeightDict[key][1] = float(row.VALUE)*0.01
    else: weightHeightDict[key] = [-1, float(row.VALUE)*0.01]

# filter out if the key only have weight or height
for key, value in list(weightHeightDict.items()):
  if value[0] == -1 or value[1] == -1:
    del weightHeightDict[key]

# Plug the weight and height to the main dataframe
for row in diagnosisDF.itertuples():
  if (row.SUBJECT_ID, row.HADM_ID, row.ADMITTIME) in weightHeightDict:
    weight, height = weightHeightDict[(row.SUBJECT_ID, row.HADM_ID, row.ADMITTIME)]
    diagnosisDF.at[row.Index, "WEIGHT"] = weight
    diagnosisDF.at[row.Index, "HEIGHT"] = height

# Drop the rows if the weight or height is NaN
diagnosisDF = diagnosisDF.dropna()


C:\Users\Steven Lu\AppData\Local\Temp\ipykernel_2916\23696007.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  CHARTEVENTS = pd.read_csv('../mimic-iii-clinical-database-1.4/CHARTEVENTS.csv.gz', compression='gzip', usecols=["SUBJECT_ID", "HADM_ID", "CHARTTIME", "ITEMID", "VALUE"]).dropna()


In [3]:
# Add the age and gender to the main dataframe by using the ["SUBJECT_ID", "GENDER", "DOB"] in PATIENTS
# SUBJECT_ID: Each patient has unique ID
# GENDER: patient's gender
# DOB: patient's date of birth

import pandas as pd

# read PATIENTS.csv.gz and change the DOB only have date
PATIENTS = pd.read_csv('../mimic-iii-clinical-database-1.4/PATIENTS.csv.gz', compression='gzip')
PATIENTS["DOB"] = pd.to_datetime(PATIENTS["DOB"]).dt.date.astype(str)

# create two dictionary with key = SUBJECT
patientBrith = dict(zip(PATIENTS["SUBJECT_ID"], PATIENTS["DOB"]))
patientGender = dict(zip(PATIENTS["SUBJECT_ID"], PATIENTS["GENDER"]))

# Plug the Gender to the main dataframe
for row in diagnosisDF.itertuples():
  diagnosisDF.at[row.Index, "GENDER"] = patientGender[row.SUBJECT_ID]

# Plug the Age to the main dataframe
for row in diagnosisDF.itertuples():
  admitYear, admitMonth, admitDate = row.ADMITTIME.split("-")
  birthYear, birthMonth, birthDate = patientBrith[row.SUBJECT_ID].split("-")
  patientMonth = int(admitYear)*12+int(admitMonth) - int(birthYear)*12+int(birthMonth)
  age, remainder = patientMonth//12, patientMonth%12
  if remainder >= 6: age += 1
  diagnosisDF.at[row.Index, "AGE"] = int(age)

# Drop the SUBJECT_ID, HADM_ID, ADMITTIME, since it is not needed anymore
diagnosisDF = diagnosisDF.drop(columns=["SUBJECT_ID", "HADM_ID", "ADMITTIME"])
print(diagnosisDF.head(10))


{249: 'F', 250: 'F', 251: 'M', 252: 'M', 253: 'F', 255: 'M', 256: 'M', 257: 'F', 258: 'F', 260: 'F', 261: 'M', 262: 'M', 263: 'M', 264: 'F', 265: 'M', 266: 'F', 267: 'F', 268: 'F', 269: 'M', 270: 'M', 663: 'F', 664: 'F', 665: 'M', 666: 'F', 667: 'F', 668: 'F', 669: 'M', 670: 'M', 671: 'M', 672: 'M', 673: 'M', 674: 'F', 675: 'F', 676: 'M', 677: 'M', 678: 'F', 679: 'F', 680: 'F', 681: 'F', 682: 'F', 685: 'F', 686: 'F', 687: 'M', 688: 'M', 689: 'F', 690: 'M', 692: 'F', 693: 'F', 694: 'F', 695: 'F', 696: 'F', 697: 'M', 698: 'F', 699: 'M', 700: 'M', 702: 'M', 703: 'M', 704: 'M', 705: 'M', 707: 'F', 708: 'M', 709: 'F', 710: 'F', 711: 'M', 712: 'M', 713: 'F', 715: 'F', 716: 'M', 717: 'F', 718: 'M', 719: 'M', 720: 'F', 721: 'M', 722: 'F', 723: 'F', 724: 'M', 725: 'M', 726: 'F', 727: 'F', 728: 'M', 729: 'F', 730: 'M', 731: 'F', 732: 'F', 733: 'M', 734: 'F', 735: 'F', 736: 'F', 737: 'F', 738: 'M', 739: 'F', 740: 'F', 741: 'M', 742: 'F', 743: 'M', 744: 'M', 745: 'M', 746: 'M', 747: 'M', 748: 'M',

C:\Users\Steven Lu\AppData\Local\Temp\ipykernel_2916\3989426574.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'M' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  diagnosisDF.at[row.Index, "GENDER"] = patientGender[row.SUBJECT_ID]


In [31]:
# Replace the typo in the data
for index, row in diagnosisDF.iterrows():
  original = row["DIAGNOSIS"]

  if original == "CORNARY ARTERY DISEASE":
    original = "CORONARY ARTERY DISEASE"
    continue
  elif original == "":
    diagnosisDF.drop(index, inplace=True)
    continue

  original = row["DIAGNOSIS"]
  while original == " " or original == "\"":
    original = original[1:]
  while original == " " or original == "\"":
    original = original[:-1]
  diagnosisDF.at[index, "DIAGNOSIS"] = original


In [32]:
diagnosisDF.to_csv('diagnosis.csv', index=False)


In [33]:
import pickle

# Do feature extraction on Diagnosis and Gender, and save it to pickle
diagnosisLabelToInt = dict([(d, index) for index, d in enumerate(set(diagnosisDF["DIAGNOSIS"]))])
diagnosisIntToLabel = dict([(index, d) for index, d in enumerate(set(diagnosisDF["DIAGNOSIS"]))])
with open('../pickleFiles/diagnosisLabelToInt.pkl', 'wb') as f:
  pickle.dump(diagnosisLabelToInt, f)
with open('../pickleFiles/diagnosisIntToLabel.pkl', 'wb') as f:
  pickle.dump(diagnosisIntToLabel, f)
print(f'Finish saving the diagnosisLabelToInt.pkl and diagnosisIntToLabel.pkl')

Finish saving the diagnosisLabelToInt.pkl and diagnosisIntToLabel.pkl


In [34]:
# Read the dictionary from pickle file
with open("../pickleFiles/diagnosisLabelToInt.pkl", "rb") as f:
  diagnosisLabelToInt = pickle.load(f)
with open("../pickleFiles/diagnosisIntToLabel.pkl", "rb") as f:
  diagnosisIntToLabel = pickle.load(f)

# Read the data
data = pd.read_csv('../dataProcess/diagnosis.csv')
for index, n in enumerate(data["DIAGNOSIS"]):
  print(index, n)
  print(index)
  print(diagnosisLabelToInt[n])
  
# data["DIAGNOSIS"] = [diagnosisLabelToInt[n] for n in data["DIAGNOSIS"]]


0 CORONARY ARTERY DISEASE
0
485
1 ACUTE CORONARY SYNDROME
1
1151
2 CORONARY ARTERY DISEASE
2
485
3 UNSTABLE ANGINA
3
1177
4 TRACHEAL STENOSIS/SDA
4
278
5 AORTIC VALVE DISEASE
5
1419
6 ACUTE MYOCARDIAL INFARCTION-SEPSIS
6
1086
7 AORTIC STENOSIS
7
1357
8 MYOCARDIAL INFARCTION
8
372
9 CAROTID STENOSIS
9
106
10 ? PULMONARY EMBOLUS WITH DESATURATION
10
407
11 FUNGAL MENINGITIS
11
1016
12 AORTIC STENOSIS
12
1357
13 V-FIB ARREST
13
1473
14 HEAD LACERATION
14
19
15 ATRIAL SEPTAL DEFECT
15
963
16 COMPLETE HEART BLOCK
16
712
17 AORTIC INSUFFICIENCY
17
668
18 NEWBORN
18
28
19 SUBARACHNOID HEMORRHAGE
19
691
20 CARDIAC ARREST
20
1273
21 HYPERTENSIVE EMERGENCY
21
1275
22 CORONARY ARTERY DISEASE
22
485
23 AORTIC STENOSIS
23
1357
24 NEWBORN
24
28
25 MR
25
433
26 SUBDURAL HEMATOMA
26
1390
27 FEVER
27
880
28 LYMPHOMA
28
283
29 LIVER CONTUSION
29
1308
30 S/P ASSAULT
30
1483
31 USA/SDA
31
23
32 NEWBORN
32
28
33 HEMORRHAGIC CVA
33
780
34 CORONARY ARTERY DISEASE
34
485
35 PATIENT FORAMEN OVALE
35
1040
36 CO